In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

import os

print(os.getcwd())


def update_working_directory():
    from pathlib import Path

    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)


update_working_directory()

In [ ]:
import numpy as np
import pandas as pd

from src.data.collect_data import *

In [ ]:
vocab = initiate_vocab()

historical_data, tries_session = initiate_session()

nb_known_words_session = 0

In [ ]:
for i in range(10):

    # Choose language
    input_language, output_language = choose_a_language(vocab)
    if input_language == -1:
        break

    # Pick a random word
    id_vocab = choose_a_word(vocab, input_language, output_language)

    # Take a guess
    your_guess = prompt_question(
        vocab,
        id_vocab,
        input_language,
        output_language,
        nb_known_words_session,
        tries_session,
    )

    # Answer analysis
    is_it_correct = (
        your_guess
        == vocab.loc[vocab["id_vocab"] == id_vocab, output_language].values[0]
    )
    is_it_another_word = your_guess in vocab[output_language].values

    # Do we want to stop?
    if your_guess == "STOP IT":
        print(
            "Your last word was '{}={}'".format(
                vocab.loc[a, input_language].values[0],
                vocab.loc[vocab["id_vocab"] == id_vocab, output_language].values[0],
            )
        )
        break

    # If correct answer
    if is_it_correct:
        if_correct()
        nb_known_words_session += 1
        write_it_again = None

    # If wrong answer
    if not is_it_correct:
        write_it_again = if_not_correct(
            vocab,
            id_vocab,
            input_language,
            output_language,
            your_guess,
            is_it_another_word,
        )

    # Add to historical data
    historical_data = add_historical_data(
        historical_data,
        vocab,
        id_vocab,
        input_language,
        output_language,
        is_it_correct,
        your_guess,
        write_it_again,
    )

    # Update vocab
    vocab = update_vocab(
        vocab, id_vocab, input_language, output_language, is_it_correct
    )

    tries_session += 1

In [ ]:
# Update and save historical data
update_historical_data(historical_data)

In [ ]:
# Update and save vocab
finalize_vocab(vocab)